In [3]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd


In [4]:
from bs4 import BeautifulSoup

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
import pymorphy2 # Морфологический анализатор.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [36]:
from scipy.stats import randint as sp_randint
from time import time

In [6]:
# отключаем warnings
import warnings

warnings.filterwarnings('ignore')

In [7]:
# cleantext = BeautifulSoup(raw_html, "lxml").text

In [8]:
df_train_raw = pd.read_csv('data/train.csv', sep='\t')

In [9]:
df_train_raw.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [10]:
df_train_raw.shape

(200000, 4)

In [11]:
df_test = pd.read_csv('data/test.csv', sep='\t')

In [12]:
df_test.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [13]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    

In [14]:
def prepare_data(dataframe):
    df = dataframe.copy()
    # clearhtml
    df['description'] = df['description'].apply(lambda x : cleanhtml(x) )
    # replace &quot to "
    df['description'] = df['description'].str.replace('&quot;','"')
    df['description'] = df['description'].str.replace('\u200b','')
    # uni name and description
    df['text'] = df['name'] + ' ' + df['description'] 
    # drop
    df = df.drop(columns=['name', 'description', 'id'], axis=1)
    return df
    

In [15]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == str:
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f

In [16]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [17]:
N = df_train_raw.shape[0]

In [18]:
%%time 
df_train = prepare_data(df_train_raw[:N])

CPU times: user 11.7 s, sys: 1.03 s, total: 12.7 s
Wall time: 12.9 s


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
target    200000 non-null int64
text      200000 non-null object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [20]:
df_tmp_test = prepare_data(df_test[:N])


In [21]:
df_tmp_test.head()

,text
0,Дизайнер-консультант мебели Обязанности: Рабо...
1,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,Менеджер по продажам Торговый Дом «Форт» это в...
3,Продавец-консультант в магазин одежды (ТЦ Волн...
4,Специалист по охране труда Обязанности: осущ...


In [22]:
df_id = df_test[:N].drop(columns=['name', 'description'], axis=1)

In [23]:
df_id.head()

,id
0,200000
1,200001
2,200002
3,200003
4,200004


In [24]:
label = 'target'
RND_SEED = 123
idx_features = df_train.columns != label

In [25]:
X = df_train.loc[:, idx_features].values.tolist()
y = df_train.loc[:, ~idx_features].values
tests = df_tmp_test["text"].values

In [26]:
texts = []
for item in X:
    texts.append(item[0])

In [27]:
texts[10]

'Backend Developer (Node.js/NoSQL) Сегодня Плэйкот - это 8 лет истории, 110 профессионалов и 3 игры в Top Grossing Facebook. Наш подход к работе прост: мы даем талантливым, увлеченным играми и ценящим качество людям возможность принимать решения самостоятельно. Многим трудно поверить, но у нас слово команды в том, что и как делать в игре, действительно является решающим.   Серверная часть эпического проекта "Age of Magic" ждет своего героя! Совсем скоро состоится глобальный запуск игры, и к этому моменту все должно быть идеально. И, конечно же, стабильно!   Что мы вам доверим:  разработку и поддержку сервера игры; продумывание и разработку сервисов и инструментов для мониторинга и технической поддержки игры.    Наши пожелания к кандидату:  опыт разработки серверной части приложения от 2-х лет; отличное знание JavaScript; опыт работы с высоконагруженными проектами; опыт работы с PostgreSQL; опыт работы с NoSQL (MongoDB/DynamoDB).    Также придутся кстати:  опыт работы с Amazon Web Servi

In [37]:
model = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('vect', CountVectorizer(
                            tokenizer=None, 
                            analyzer="word",
                        )),
                        ('tfidf', TfidfTransformer(
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.5,
            }) # end FeatureUnion
        ),# end union
        ('clf', MultinomialNB()),
]) # end pipeline

model_2 = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('tfidf', TfidfVectorizer(
                            min_df=5,
                            max_df=.95,
                            tokenizer=None,
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.8,
            }) # end FeatureUnion
        ),# end union
        ('clf', LogisticRegression(
            penalty='l2', 
            class_weight='balanced', 
            random_state=123, 
            )),
]) # end pipeline


# model 3 
# http://scikit-learn.org/stable/auto_examples/model_selection/\
# plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

#https://www.kaggle.com/nikitabu/tfidf-lightgbm-pipeline-with-randomsearchcv/code
#
n_estimators=20

clf = RandomForestClassifier(n_estimators=n_estimators)

model_3 = Pipeline([
        ('union', FeatureUnion(
                # Use FeatureUnion to combine the features from subject and body
            transformer_list=[
                    # Pipeline for standard bag-of-words model
                ('text', Pipeline([
                        ('tfidf', TfidfVectorizer(
                            min_df=5,
                            max_df=.95,
                            tokenizer=None,
                            norm=None, 
                            smooth_idf=False,
                        )),
                    ])
                ),
            ],# end  transformer_list
                # weight components in FeatureUnion
            transformer_weights={
                'text': 0.8,
            }) # end FeatureUnion
        ),# end union
        ('clf', clf),
]) # end pipeline


In [45]:
n_iter_search = 20
n_folds = 2 # Define number of CV folds

param_dist = {"clf__max_depth": [3, None],
              "clf__max_features": sp_randint(1, 11),
              "clf__min_samples_split": sp_randint(2, 11),
              "clf__min_samples_leaf": sp_randint(1, 11),
              "clf__bootstrap": [True, False],
              "clf__criterion": ["gini", "entropy"]}

random_search = RandomizedSearchCV(
    model_3, 
    param_distributions=param_dist, 
    cv=n_folds,
    scoring='roc_auc',
    n_iter=n_iter_search,
    n_jobs=4)

start = time()
random_search.fit(texts, y)


KeyboardInterrupt: 

In [ ]:
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [26]:
%%time 
# Схема кросс-валидации
n_splits = 5
cv = StratifiedKFold(
        n_splits=n_splits, shuffle=True, 
        random_state=RND_SEED)

scores = cross_val_score(
    model_2, texts, y,
    scoring='roc_auc', 
    cv=cv, 
n_jobs=-1)

CPU times: user 7.61 s, sys: 16.3 s, total: 23.9 s
Wall time: 31min 35s


In [27]:
scores.mean(), scores.std()

(0.9877820541888596, 0.000775901042588291)

In [28]:
model_2.fit(texts, y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max...lty='l2', random_state=123,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [29]:
predictions = model_2.predict(tests)

In [30]:
df_id['target'] = pd.DataFrame(predictions.tolist())

In [31]:
df_id.shape

(170179, 2)

In [32]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d_%H-%M')
df_id.to_csv(path_or_buf='./submission/submission_{today}.cvs'.format(today=today), index=False, sep=',')